In [2]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import math
import re

https://m.land.naver.com/search/result/양천구신월동

In [6]:
#임의의 btm, lft, top, rgt값 계산

def calculate_bounding_box(lat, lon, zoom):
    # Earth's radius in meters
    R = 6378137
    # Approximate bounding box in meters for zoom level 14 (usually covers around 1km)
    bounding_box_half_side_length = 1000  # 1km / 2

    # Convert latitude and longitude from degrees to radians
    lat_rad = math.radians(lat)
    lon_rad = math.radians(lon)
    
    # Offset calculations
    lat_offset = (bounding_box_half_side_length / R) * (180 / math.pi)
    lon_offset = (bounding_box_half_side_length / (R * math.cos(lat_rad))) * (180 / math.pi)
    
    # Calculate bounding box coordinates
    btm = lat - lat_offset
    lft = lon - lon_offset
    top = lat + lat_offset
    rgt = lon + lon_offset
    
    return btm, lft, top, rgt

In [3]:

keyword = "양천구신월동"
url = "https://m.land.naver.com/search/result/{}".format(keyword)

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Referer': 'https://m.land.naver.com/'
}

res = requests.get(url, headers=headers)
res.raise_for_status()

soup = str(BeautifulSoup(res.text, 'lxml'))

# filter 값 추출
value = soup.split("filter: {")[1].split("}")[0].replace(" ","").replace("'","")

print(value)

lat = (float)(value.split("lat:")[1].split(",")[0])
lon = (float)(value.split("lon:")[1].split(",")[0])
z = (int)(value.split("z:")[1].split(",")[0])
cortarNo = value.split("cortarNo:")[1].split(",")[0]
rletTpCds = value.split("rletTpCds:")[1].split(",")[0]
tradTpCds = value.split("tradTpCds:")[1].split()[0]


#임의의 btm, lft, top, rgt값 계산
btm, lft, top, rgt = calculate_bounding_box(lat, lon, z)
print(f"btm: {btm}, lft: {lft}, top: {top}, rgt: {rgt}")


# clusterList URL
clusterList_URL = "https://m.land.naver.com/cluster/clusterList?view=atcl&cortarNo={}&rletTpCd={}&tradTpCd={}&z={}&lat={}&lon={}&btm={}&lft={}&top={}&rgt={}"\
    .format(cortarNo, rletTpCds, tradTpCds, z, lat, lon, btm, lft, top, rgt)
print(clusterList_URL)

# clusterList URL로 GET 요청
res2 = requests.get(clusterList_URL, headers=headers)
res2.raise_for_status()
# print(res2.text)

# 데이터 파싱
try:
    data = res2.json()
    # print(json.dumps(data, indent=2, ensure_ascii=False))
except json.JSONDecodeError as e:
    print("JSON 데이터를 파싱할 수 없습니다:", e)
    print("응답 내용:", res2.text)
    
values = data['data']['ARTICLE']
# print(values)

result_list = []
# 큰 원으로 구성되어 있는 전체 매물그룹(values)을 load 하여 한 그룹씩 세부 쿼리 진행
for v in values:
    lgeo = v['lgeo']
    count = v['count']
    z2 = v['z']
    lat2 = v['lat']
    lon2 = v['lon']

    len_pages = math.ceil(count / 20) + 1
    for idx in range(1, len_pages):

        # articleList_URL    
        articleList_URL = "https://m.land.naver.com/cluster/ajax/articleList?""itemId={}&mapKey=&lgeo={}&showR0=&" \
               "rletTpCd={}&tradTpCd={}&z={}&lat={}&""lon={}&totCnt={}&cortarNo={}&page={}"\
            .format(lgeo, lgeo, rletTpCds, tradTpCds, z2, lat2, lon2, count,cortarNo, idx)
        # print(articleList_URL)
        res3 = requests.get(articleList_URL, headers=headers)
        res3.raise_for_status()
                
        try:
            data2 = res3.json()
            articles = data2['body']
            for article in articles:
                result_list.append(article)
        except json.JSONDecodeError as e:
            print("JSON 데이터를 파싱할 수 없습니다:", e)
            print("응답 내용:", res3.text)

#변환 후 저장
df = pd.DataFrame(result_list)
df.to_csv(f'{keyword}_list.csv', index=False, encoding='utf-8-sig')
print("CSV 파일로 저장되었습니다.")
            
        


lat:37.527925,
lon:126.8342,
z:14,
cortarNo:1147010300,
cortarNm:신월동,
rletTpCds:*,
tradTpCds:A1:B1:B2

btm: 37.51894184715881, lft: 126.82287274611544, top: 37.5369081528412, rgt: 126.84552725388455
https://m.land.naver.com/cluster/clusterList?view=atcl&cortarNo=1147010300&rletTpCd=*&tradTpCd=A1:B1:B2&z=14&lat=37.527925&lon=126.8342&btm=37.51894184715881&lft=126.82287274611544&top=37.5369081528412&rgt=126.84552725388455
JSON 데이터를 파싱할 수 없습니다: Expecting value: line 1 column 1 (char 0)
응답 내용: <!DOCTYPE HTML>
<html lang="ko">
<head>
    <link rel='shortcut icon' href='https://m.naver.com/favicon.ico' type='image/x-icon'>
    <meta name="format-detection" content="telephone=no">
    <meta http-equiv="Content-Type" content="text/html;charset=utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no">
    <meta name="format-detection" content="telephone=no">
    <title>네이버페이 부동산</title>
    <meta http-equiv="Conte

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 49 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   atclNo               600 non-null    object 
 1   cortarNo             600 non-null    object 
 2   atclNm               600 non-null    object 
 3   atclStatCd           600 non-null    object 
 4   rletTpCd             600 non-null    object 
 5   uprRletTpCd          600 non-null    object 
 6   rletTpNm             600 non-null    object 
 7   tradTpCd             600 non-null    object 
 8   tradTpNm             600 non-null    object 
 9   vrfcTpCd             600 non-null    object 
 10  flrInfo              600 non-null    object 
 11  prc                  600 non-null    int64  
 12  rentPrc              600 non-null    int64  
 13  hanPrc               600 non-null    object 
 14  spc1                 600 non-null    object 
 15  spc2                 600 non-null    obj

In [13]:
df.head()

,atclNo,cortarNo,atclNm,atclStatCd,rletTpCd,uprRletTpCd,rletTpNm,tradTpCd,tradTpNm,vrfcTpCd,...,etRoomCnt,tradePriceHan,tradeRentPrice,tradeCheckedByOwner,cpLinkVO,dtlAddrYn,dtlAddr,sameAddrMaxPrc2,sameAddrMinPrc2,sellrNm
0,2425857390,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,B1,전세,MOBL,...,0,,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,,NaN,NaN,NaN
1,2425798538,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,A1,매매,MOBL,...,0,,0,False,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",N,,NaN,NaN,NaN
2,2426086256,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,B1,전세,OWNER,...,0,,0,False,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",N,,NaN,NaN,NaN
3,2426074781,1147010300,목동센트럴아이파크위브,R0,A01,A01,아파트,A1,매매,OWNER,...,0,,0,False,"{'cpId': 'sunbang', 'mobileArticleLinkTypeCode...",N,,NaN,NaN,NaN
4,2426078377,1147010300,스카이캐슬,R0,A01,A01,아파트,B1,전세,OWNER,...,0,,0,False,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",N,,NaN,NaN,NaN


In [51]:
df1 = df['cpCnt'].unique()
df1

array([3, 8, 4, 5, 6, 2, 7, 1, 9], dtype=int64)

In [43]:
df1 = df['cpLinkVO'].unique()
df1

array(["{'cpId': 'sunbang', 'mobileArticleLinkTypeCode': 'NONE', 'mobileBmsInspectPassYn': 'Y', 'pcArticleLinkUseAtArticleTitle': False, 'pcArticleLinkUseAtCpName': False, 'mobileArticleLinkUseAtArticleTitle': False, 'mobileArticleLinkUseAtCpName': False}",
       "{'cpId': 'bizmk', 'mobileArticleLinkTypeCode': 'NONE', 'mobileBmsInspectPassYn': 'Y', 'pcArticleLinkUseAtArticleTitle': False, 'pcArticleLinkUseAtCpName': False, 'mobileArticleLinkUseAtArticleTitle': False, 'mobileArticleLinkUseAtCpName': False}",
       "{'cpId': 'NEONET', 'mobileArticleLinkTypeCode': 'NONE', 'mobileBmsInspectPassYn': 'Y', 'pcArticleLinkUseAtArticleTitle': False, 'pcArticleLinkUseAtCpName': False, 'mobileArticleLinkUseAtArticleTitle': False, 'mobileArticleLinkUseAtCpName': False}",
       "{'cpId': 'asil', 'mobileArticleUrl': 'https://asil.kr/app/sale_info.jsp?UID=', 'mobileArticleLinkTypeCode': 'ALL', 'mobileBmsInspectPassYn': 'Y', 'pcArticleLinkUseAtArticleTitle': False, 'pcArticleLinkUseAtCpName': False,

In [8]:
df.isnull().sum()

atclNo                   0
cortarNo                 0
atclNm                   0
atclStatCd               0
rletTpCd                 0
uprRletTpCd              0
rletTpNm                 0
tradTpCd                 0
tradTpNm                 0
vrfcTpCd                 0
flrInfo                  0
prc                      0
rentPrc                  0
hanPrc                   0
spc1                     0
spc2                     0
direction               10
atclCfmYmd               0
repImgUrl              457
repImgTpCd             457
repImgThumb            457
lat                      0
lng                      0
atclFetrDesc            44
tagList                  0
bildNm                   0
minute                   0
sameAddrCnt              0
sameAddrDirectCnt        0
sameAddrHash           370
sameAddrMaxPrc         370
sameAddrMinPrc         370
cpid                     0
cpNm                     0
cpCnt                    0
rltrNm                   0
directTradYn             0
m

## 데이터 전처리 

In [162]:
df.drop_duplicates()
df

,atclNo,cortarNo,atclNm,atclStatCd,rletTpCd,uprRletTpCd,rletTpNm,tradTpCd,tradTpNm,vrfcTpCd,...,dtlAddr,sameAddrPremMin,sameAddrPremMax,repImgUrl,repImgTpCd,repImgThumb,sameAddrMaxPrc2,sameAddrMinPrc2,town,sellrNm
0,2426329376,1168010300,디에이치퍼스티어아이파크,R0,A01,A01,아파트,B1,전세,NDOC1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,강남구 개포동,NaN
1,2426319116,1168010300,디에이치퍼스티어아이파크,R0,A01,A01,아파트,A1,매매,NDOC1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,강남구 개포동,NaN
2,2426330636,1168010300,디에이치퍼스티어아이파크,R0,A01,A01,아파트,B1,전세,NDOC1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,강남구 개포동,NaN
3,2426292674,1168010300,래미안블레스티지,R0,A01,A01,아파트,B1,전세,OWNER,...,NaN,NaN,NaN,/20240529_150/land_naver_1716954134244cK1vF_JP...,10,f130_98,NaN,NaN,강남구 개포동,NaN
4,2426303207,1168010300,디에이치퍼스티어아이파크,R0,A01,A01,아파트,B2,월세,NDOC1,...,NaN,NaN,NaN,NaN,NaN,NaN,400,500,강남구 개포동,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24762,2422527668,1168010100,대명벨리온(도시형),R0,A01,A01,아파트,B2,월세,OWNER,...,NaN,NaN,NaN,/20240516_58/land_naver_1715846838107XrkpK_JPE...,10,f130_98,NaN,NaN,강남구 역삼동,NaN
24763,2425514914,1168010100,뜨라네(828-76),R0,A01,A01,아파트,B2,월세,DOC,...,NaN,NaN,NaN,NaN,NaN,NaN,260,300,강남구 역삼동,NaN
24764,2422566461,1168010100,중소형사무실,R0,D01,D01,사무실,B2,월세,OWNER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,강남구 역삼동,NaN
24765,2422537172,1168010100,네스빌,R0,A02,A02,오피스텔,B1,전세,NDOC1,...,NaN,NaN,NaN,/20240508_101/land_naver_1715153902606uqoUF_JP...,10,f130_98,NaN,NaN,강남구 역삼동,NaN


In [116]:
selected_columns = ["atclNo", "atclNm", "rletTpNm","tradTpNm", "flrInfo", "prc","rentPrc","hanPrc","spc1","spc2", "direction", "lat", "lng", "atclFetrDesc", "tagList","bildNm","town"]
df = df[selected_columns].reset_index(drop=True)

In [154]:
df['id'] = range(1, len(df) + 1)

In [122]:
apartment_df = df[df['rletTpNm'] == '아파트']
officetel_df = df[df['rletTpNm'] == '오피스텔']

In [136]:
apartment_df

,atclNo,atclNm,tradTpNm,flrInfo,prc,rentPrc,hanPrc,spc1,spc2,direction,lat,lng,atclFetrDesc,tagList,bildNm,town,area,id
0,2426329376,디에이치퍼스티어아이파크,전세,11/32,320000,0,32억,220,171.65,남서향,37.481919,127.057858,66 O로얄동로얄층O조중석식O최대규모커뮤니티O근린공원O구룡역근접,"['2년이내', '대단지', '대형평수', '방네개이상']",104동,강남구 개포동,1,1
1,2426319116,디에이치퍼스티어아이파크,매매,21/26,430000,0,43억,144,112.85,남서향,37.478823,127.058843,44 뷰가 확 트여야 가치가 더 높습니다 입주가능한 매매 물건,"['2년이내', '대단지', '대형평수', '방네개이상']",146동,강남구 개포동,2,2
2,2426330636,디에이치퍼스티어아이파크,전세,12/35,250000,0,25억,169,132.81,남동향,37.480453,127.058339,51O뷰좋은집O커뮤니티바로연결O근린공원O생활편리동,"['2년이내', '대단지', '대형평수', '방네개이상']",101동,강남구 개포동,3,3
3,2426292674,래미안블레스티지,전세,4/8,105000,0,"10억 5,000",80,59.88,남서향,37.480127,127.063991,24 V풀옵션 V장기거주 V호텔식 조중식 V수영장사우나,"['10년이내', '융자금적은', '대단지']",223동,강남구 개포동,4,4
4,2426303207,디에이치퍼스티어아이파크,월세,22/33,100000,600,10억,201,156.96,남향,37.481182,127.057478,61O시그니처동O커뮤니티연결동O근린공원O뷰좋음,"['2년이내', '대단지', '대형평수', '방네개이상']",103동,강남구 개포동,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24759,2423623584,강남서해그랑블(주상복합),월세,중/10,40000,80,4억,94,70.57,남서향,37.493134,127.039216,공동불가 입점업소 2룸2욕실 뷰 컨디션 좋은호실 빠른계약예상,"['15년이내', '방두개', '화장실두개']",1동,강남구 역삼동,24760,5760
24761,2425505199,금호어울림,월세,중/15,40000,300,4억,155,133.77,남향,37.492233,127.034715,채광좋은남향 넓은평수 내부깨끗 주차편리 입주협의가능해요,"['25년이내', '융자금없는', '대형평수', '방네개이상']",101동,강남구 역삼동,24762,5761
24762,2422527668,대명벨리온(도시형),월세,3/7,1000,95,"1,000",27,17.68,북서향,37.497964,127.039165,"BEST역삼역도보10분이내,보안굿","['15년이내', '역세권', '소형평수']",1동,강남구 역삼동,24763,5762
24763,2425514914,뜨라네(828-76),월세,저/6,10000,260,1억,97,84.42,동향,37.497454,127.033053,쓰리룸 리모델링호실 빠른입주가능 내부깨끗 주차편리,"['25년이내', '융자금없는', '방세개', '화장실두개']",A동,강남구 역삼동,24764,5763


In [147]:
officetel_df.to_csv('GangnamOfficetel.csv', index=False, encoding='utf-8-sig')

In [111]:
filtered_df = df[~df['rletTpNm'].isin(['아파트', '오피스텔'])]

In [142]:
dfforadd = df

In [149]:
selected_columns = ["lat", "lng", "area"]
dfforadd = dfforadd[selected_columns]

In [189]:
cols = ['id'] + [col for col in df.columns if col != 'id']
df = df[cols]

In [191]:
df = df.drop(columns=['town'])

In [192]:
df.to_csv('GangnamApartment.csv', index=False, encoding='utf-8-sig')

In [173]:
df.isnull().sum()

id              0
area            0
atclNo          0
atclNm          0
rletTpNm        0
tradTpNm        0
flrInfo         0
prc             0
rentPrc         0
hanPrc          0
spc1            0
spc2            0
direction       0
atclFetrDesc    0
tagList         0
bildNm          0
dtype: int64

In [171]:
df = df.fillna(0)

In [174]:
df.to_csv(f'GangnamALL.csv', index=False, encoding='utf-8-sig')

In [88]:
df.info() # 데이터프레임의 정보 출력

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   atclNo    100 non-null    int64  
 1   atclNm    100 non-null    object 
 2   rletTpNm  100 non-null    object 
 3   prc       100 non-null    int64  
 4   lat       100 non-null    float64
 5   lng       100 non-null    float64
dtypes: float64(2), int64(2), object(2)
memory usage: 4.8+ KB


In [90]:
df.isnull().sum() # 데이터프레임의 결측치 정보 출력

atclNo      0
atclNm      0
rletTpNm    0
prc         0
lat         0
lng         0
dtype: int64

In [7]:
df = pd.read_csv('양천구_신월동_100.csv',encoding='utf-8')
df

,atclNo,atclNm,rletTpNm,prc,lat,lng
0,2426130767,목동센트럴아이파크위브,아파트,60000,37.517669,126.844850
1,2425438083,상가주택,상가주택,250000,37.517523,126.842308
2,2423609940,스카이캐슬,아파트,97000,37.520542,126.844645
3,2425722784,단독,단독/다가구,15000,37.518080,126.842841
4,2422558122,일반상가,상가,108000,37.517721,126.842002
...,...,...,...,...,...,...
95,2423233490,빌라,빌라,25500,37.521391,126.844122
96,2426060226,신성미소지움3차,아파트,63000,37.522890,126.841696
97,2425566480,금용,아파트,83000,37.520768,126.842091
98,2426059502,신정뉴타운롯데캐슬,아파트,81000,37.518923,126.847877


## 데이터 밸런스 맞추기

In [24]:
df = pd.read_csv('GangnamAdd.csv',encoding='utf-8')
df1 = pd.read_csv('GangnamALL.csv',encoding='utf-8')

In [21]:
df

,area,lat,lng,road_address,address
0,1,37.481919,127.057858,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
1,2,37.478823,127.058843,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
2,3,37.480453,127.058339,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
3,4,37.480127,127.063991,서울특별시 강남구 선릉로 8,서울 강남구 개포동 1280
4,5,37.481182,127.057478,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
...,...,...,...,...,...
24762,24763,37.497964,127.039165,서울특별시 강남구 역삼로25길 27,서울 강남구 역삼동 727-2
24763,24764,37.497454,127.033053,서울특별시 강남구 개포로 310,서울 강남구 역삼동 828-76
24764,24765,37.493709,127.038101,서울특별시 강남구 논현로67길 23,서울 강남구 역삼동 795-2
24765,24766,37.495225,127.032596,서울특별시 강남구 역삼로7길 17,서울 강남구 역삼동 830-41


In [25]:
df = df1.sample(frac=1).reset_index(drop=True)
df

,id,area,atclNo,atclNm,rletTpNm,tradTpNm,flrInfo,prc,rentPrc,hanPrc,spc1,spc2,direction,atclFetrDesc,tagList,bildNm
0,20374,20374,2423542341,대형사무실,사무실,월세,1/5,25000,2300,"2억 5,000",561,561.31,남서향,0,"['10년이내', '1층', '주차가능', '천장에어컨']",0
1,20938,20938,2425941602,빌라,빌라,매매,2/2,85000,0,"8억 5,000",48,44.28,남향,0,"['25년이상', '역세권', '소형평수']",0
2,11749,11749,2425945043,중소형사무실,사무실,월세,B1/5,2000,160,"2,000",89,89.00,남향,스튜디오 추천,"['25년이내', '융자금없는', '지하층', '관리비10만원이하']",0
3,17651,17651,2424576763,푸른마을,아파트,전세,3/15,75000,0,"7억 5,000",103,84.93,남향,"주인직매물,조용일조권좋은남향,권리깨끗","['25년이상', '역세권', '방세개', '화장실두개']",102동
4,16198,16198,2422550993,미성2차,아파트,매매,14/17,300000,0,30억,95,74.40,동향,실매물 투자가치 유망한곳으로 매수적기 입니다,"['25년이상', '방세개', '화장실한개', '고층']",28동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24762,14031,14031,2422035727,기타,원룸,월세,중/3,1000,90,"1,000",30,30.00,동향,신사역 가로수길 주차 가능한 원룸 월세,"['25년이상', '소형평수', '방한개']",0
24763,304,304,2424823564,디에이치퍼스티어아이파크,아파트,월세,8/21,30000,420,3억,110,84.82,남동향,33A 생활인프라 모두 근접한 로얄세대 보증금조정가능,"['2년이내', '융자금없는', '역세권', '대단지']",113동
24764,20152,20152,2424016436,청담아이파크,아파트,월세,중/19,120000,330,12억,137,110.13,동향,내부 에이급 리모델링 최상 한강뷰 멋진집입니다.,"['15년이내', '대형평수', '방네개이상', '화장실두개']",1동
24765,1023,1023,2426280490,디에이치퍼스티어아이파크,아파트,월세,중/26,60000,300,6억,110,84.82,남향,33 전망좋은 동 학세권 최고 커뮤니티 신축,"['2년이내', '대단지', '방세개', '화장실두개']",159동


In [27]:
df.to_csv('GangnamALL.csv', index=False, encoding='utf-8-sig')

In [ ]:
df

In [6]:
df

,area,lat,lng,road_address,address
0,1,37.481919,127.057858,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
1,2,37.478823,127.058843,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
2,3,37.480453,127.058339,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
3,4,37.480127,127.063991,서울특별시 강남구 선릉로 8,서울 강남구 개포동 1280
4,5,37.481182,127.057478,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1
...,...,...,...,...,...
24762,24763,37.497964,127.039165,서울특별시 강남구 역삼로25길 27,서울 강남구 역삼동 727-2
24763,24764,37.497454,127.033053,서울특별시 강남구 개포로 310,서울 강남구 역삼동 828-76
24764,24765,37.493709,127.038101,서울특별시 강남구 논현로67길 23,서울 강남구 역삼동 795-2
24765,24766,37.495225,127.032596,서울특별시 강남구 역삼로7길 17,서울 강남구 역삼동 830-41


In [7]:
df1

,id,area,atclNo,atclNm,rletTpNm,tradTpNm,flrInfo,prc,rentPrc,hanPrc,spc1,spc2,direction,atclFetrDesc,tagList,bildNm
0,1,1,2426329376,디에이치퍼스티어아이파크,아파트,전세,11/32,320000,0,32억,220,171.65,남서향,66 O로얄동로얄층O조중석식O최대규모커뮤니티O근린공원O구룡역근접,"['2년이내', '대단지', '대형평수', '방네개이상']",104동
1,2,2,2426319116,디에이치퍼스티어아이파크,아파트,매매,21/26,430000,0,43억,144,112.85,남서향,44 뷰가 확 트여야 가치가 더 높습니다 입주가능한 매매 물건,"['2년이내', '대단지', '대형평수', '방네개이상']",146동
2,3,3,2426330636,디에이치퍼스티어아이파크,아파트,전세,12/35,250000,0,25억,169,132.81,남동향,51O뷰좋은집O커뮤니티바로연결O근린공원O생활편리동,"['2년이내', '대단지', '대형평수', '방네개이상']",101동
3,4,4,2426292674,래미안블레스티지,아파트,전세,4/8,105000,0,"10억 5,000",80,59.88,남서향,24 V풀옵션 V장기거주 V호텔식 조중식 V수영장사우나,"['10년이내', '융자금적은', '대단지']",223동
4,5,5,2426303207,디에이치퍼스티어아이파크,아파트,월세,22/33,100000,600,10억,201,156.96,남향,61O시그니처동O커뮤니티연결동O근린공원O뷰좋음,"['2년이내', '대단지', '대형평수', '방네개이상']",103동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24762,24763,24763,2422527668,대명벨리온(도시형),아파트,월세,3/7,1000,95,"1,000",27,17.68,북서향,"BEST역삼역도보10분이내,보안굿","['15년이내', '역세권', '소형평수']",1동
24763,24764,24764,2425514914,뜨라네(828-76),아파트,월세,저/6,10000,260,1억,97,84.42,동향,쓰리룸 리모델링호실 빠른입주가능 내부깨끗 주차편리,"['25년이내', '융자금없는', '방세개', '화장실두개']",A동
24764,24765,24765,2422566461,중소형사무실,사무실,월세,2/6,2000,200,"2,000",129,129.60,남서향,사진O 층전체사용 주차가능 가성비 좋은 사무실,"['25년이상', '지상층(1층제외)', '관리비10만원이하', '주차가능']",0
24765,24766,24766,2422537172,네스빌,오피스텔,전세,2/9,19000,0,"1억 9,000",44,23.68,남동향,실물 더 넓은 테라스 굿 시세대비 저렴 빠른계약예상,"['25년이내', '테라스', '역세권']",1동


In [12]:
df = df.sample(frac=1).reset_index(drop=True)
df

,area,lat,lng,road_address,address,id,atclNo,atclNm,rletTpNm,tradTpNm,flrInfo,prc,rentPrc,hanPrc,spc1,spc2,direction,atclFetrDesc,tagList,bildNm
0,23791,37.497110,127.037435,서울특별시 강남구 역삼로19길 29,서울 강남구 역삼동 745-12,23791,2425751350,일반상가,상가,월세,1/5,1000,220,"1,000",76,76.00,동향,좋은 매물만 엑기스로 추려드리겠습니다.,"['25년이상', '역세권', '1층', '관리비20만원이하']",0
1,22531,37.485625,127.035919,서울특별시 강남구 강남대로42길 23,서울 강남구 도곡동 954-18,22531,2420975917,일반상가,상가,월세,1/6,11000,980,"1억 1,000",250,132.00,남향,원하시는 매장 반드시 찾아 드리겠습니다.,"['2년이내', '융자금없는', '역세권', '1층']",0
2,17726,37.481545,127.078599,서울특별시 강남구 광평로10길 6,서울 강남구 일원동 731,17726,2421964775,한솔마을,아파트,매매,4/5,230000,0,23억,110,84.73,남향,확장 샤시교체 특올수리 입주가능 강력추천드립니다,"['25년이상', '올수리', '방세개', '화장실두개']",303동
3,893,37.479332,127.058767,서울특별시 강남구 개포로 310,서울 강남구 개포동 660-1,893,2426321631,디에이치퍼스티어아이파크,아파트,월세,중/35,90000,540,9억,169,132.81,남동향,0,"['2년이내', '대단지', '대형평수', '방네개이상']",173동
4,17234,37.490251,127.082356,서울특별시 강남구 양재대로31길 13,서울 강남구 일원동 686-3,17234,2421849895,단독,단독/다가구,월세,B1/2,2000,60,"2,000",40,40.00,남동향,"2룸 단독주택 반지하 월세, 보증금 조절 가능, 미닫이문 있음","['25년이상', '융자금없는', '소형평수', '방두개']",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24762,23969,37.494933,127.035423,서울특별시 강남구 역삼로 149,서울 강남구 역삼동 751-7,23969,2423595052,중소형사무실,사무실,월세,1/5,23000,2300,"2억 3,000",732,732.20,남향,대로변 소형빌딩 임대 현병원운영중 원복협의,"['25년이상', '1층', '주차가능', '엘리베이터']",0
24763,22567,37.491811,127.040029,서울특별시 강남구 도곡로 188,서울 강남구 도곡동 544-2,22567,2420911223,일반상가,상가,월세,3/4,5000,300,"5,000",198,181.80,북서향,"강남BEST 도곡로대로변,가시성,노출도 좋은상가,시세대비 저렴한 월세","['25년이상', '고층', '주차가능', '총4층']",0
24764,24238,37.498924,127.033882,서울특별시 강남구 테헤란로16길 9,서울 강남구 역삼동 735-29,24238,2425134432,중소형사무실,사무실,월세,1/5,4000,600,"4,000",133,133.80,서향,실사진O 역삼역 S급 컨디션 인테리어 손 볼 곳 없는 사무실,"['25년이상', '역세권', '1층', '주차가능']",0
24765,14223,37.519372,127.019631,서울특별시 강남구 도산대로1길 40,서울 강남구 신사동 507-11,14223,2421547169,대형사무실,사무실,월세,3/6,4000,400,"4,000",200,160.00,서향,D실사진. 더블역세권 인테리어 냉난방기 완비. 자주식주차,"['25년이상', '역세권', '중층', '주차가능']",0


In [8]:
merged_df = pd.merge(df, df1, on='area')

In [10]:
rletTpNm_distribution = merged_df['rletTpNm'].value_counts()

In [ ]:
rletTpNm_distribution = df1['rletTpNm'].value_counts()

In [11]:
rletTpNm_distribution

rletTpNm
사무실        6911
아파트        5764
상가         4919
빌라         2138
건물         1372
단독/다가구     1272
오피스텔        925
재건축         855
원룸          412
상가주택        124
토지           52
오피스텔분양권      13
아파트분양권        8
공장/창고         2
Name: count, dtype: int64